<a href="https://colab.research.google.com/github/AndreaMorbello/Progetto-IA2/blob/main/Notebooks/Notebook-IA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook progetto baseline

## 1. Introduzione al progetto

### 1.1. Obiettivo del modello

L'obiettivo di questo progetto è quello di sviluppare un modello in grado di prevedere il rating ELO di un giocatore (in particolare del giocatore con i pezzi bianchi) a partire da feature estratte da una sua partita.
Questo modello, che chiameremo modello baseline, sarà poi confrontato con un secondo modello ... [DA COMPLETARE]


### 1.2. Presentazione del dataset

Il dataset che abbiamo utilizzato proviene da Kaggle:
https://www.kaggle.com/datasets/datasnaek/chess/code/data
Contiene 20058 partite di scacchi e contiene:
- id (Stringa alfanumerica identificativa della partita);
- rated (La partita è competitiva e comportarà una variazione dell'ELO del giocatore? True o False);
- created_at (Momento di inizio della partita in unix time);
- last_move_at (Momento di fine della partita in unix time);
- turns (Numero di turni di gioco intesi come coppie di mosse bianco-nero)
- victory_status (Come la partita si è conclusa: Scacco matto, stallo, tempo esaurito...);
- winner (Colore del giocatore che ha vinto la partita o draw (patta));
- increment_code (Minuti assegnati a ciascun giocatore all'inizio della partita  secondi assegnati dopo ciascuna mossa);
- white_id (Nome utente del giocatore bianco);
- white_rating (Punteggio ELO del giocatore bianco);
- black_id (Nome utente del giocatore nero);
- black_rating (Punteggio ELO del giocatore nero);
- moves (Tutte le mosse della partita in notazione scacchistica);
- opening_eco (codice identificativo dell'apertura della partita);
- opening_name (nome dell'apertura);
- opening_ply (singole mosse appartenenti alla fase di apertura della partita);

### 1.3. Importazione del Dataset




In [2]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
DATASET_PATH = DATA_DIR / "games.csv"

if not DATASET_PATH.is_file():
    print(f"Dataset non trovato in: {DATASET_PATH}")

Dataset non trovato in: /data/games.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


df=pd.read_csv('games.csv')
df.info()

# Iniziamo subito a pulire i dati:
# verifichiamo anzitutto se sono presenti duplicati di stesse partite

duplicates = len(df[df['id'].duplicated()])
print('Ci sono: '+str(duplicates)+' duplicati')

# Prima di rimuovere i duplicati verifichiamo che le partite con id uguale abbiano
# effettivamente valori identici nel dataset

id_dup=df['id'][df['id'].duplicated()]
print(id_dup)

# Ora possiamo eliminare dal dataframe le colonne che sicuramente
# non ci aiuteranno in nessun modo: Gli ID dei giocatori e della partita,
# terremo l'Opening Eco invece che l'Opening Name in quanto più consistente
# e unificato (variazioni della stessa apertura ricadono sotto la stessa categoria).
# Infine rimuoviamo la colonna delle mosse giocate: ovviamente valutare
# una partita, e di conseguenza il livello dei giocatori è un compito che
# si basa solitamente proprio sulle mosse giocate. Quello che vogliamo
# verificare noi, invece, è l'accuratezza con cui si può predire il livello
# di un giocatore avendo solo il rating dell'avversario e informazioni laterali
# sulla partita.

df=df.drop(columns=['white_id','black_id','id','opening_name','moves'])

#Ora convertiamo le colonne di inizio e fine partita, che sono in formato Unix time, in formato datetime
#Dopodichè inseriremo una più semplice colonna di durata partita al posto delle due colonne iniziali

df_times = df[['created_at','last_move_at']].copy()

df_times['created_at_dt'] = pd.to_datetime(df_times['created_at']/1000, unit='s', origin='unix')

df_times['last_move_at_dt'] = pd.to_datetime(df_times['last_move_at']/1000, unit='s', origin='unix')

df_times['created_time'] = df_times['created_at_dt'].dt.time
df_times['last_move_time'] = df_times['last_move_at_dt'].dt.time
df_times['duration_seconds'] = (df_times['last_move_at_dt'] - df_times['created_at_dt']).dt.total_seconds()

#Adesso che abbiamo la durata della partita in secondi, inseriamo la colonna nel dataframe e rimuoviamo quelle iniziali

df=df.drop(columns=['created_at','last_move_at'])
df=df.join(df_times['duration_seconds'])

#Ora invece ci occupiamo della colonna 'increment_code', che è della forma "X+Y"
#dove X sono i minuti concessi a ciascun giocatore per giocare le proprie mosse e Y
#è il tempo che gli viene riaggiunto una volta giocata la mossa.
#Vogliamo separare questi due valori e tenerli in colonne separate

#Split di increment_code in due colonne
df[['base_time', 'increment_time']] = (
    df['increment_code'].str.split('+', expand=True).astype(int))

#Rimuoviamo la colonna originale
df = df.drop(columns=['increment_code'])

df.tail(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  object 
 15  opening_ply     20058 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 2.3+ MB
Ci sono: 9

,rated,turns,victory_status,winner,white_rating,black_rating,opening_eco,opening_ply,duration_seconds,base_time,increment_time
20008,True,22,resign,white,2255,2008,C50,6,609.493,60,30
20009,True,62,resign,black,2007,2249,E17,14,4675.717,30,30
20010,True,70,resign,black,1801,2246,E38,8,2686.126,20,20
20011,True,63,draw,draw,2239,2401,B23,5,4804.962,45,45
20012,True,58,resign,black,1471,2239,C53,12,2169.851,20,20
20013,True,95,resign,black,2094,2228,C60,6,8755.960,45,45
20014,True,109,draw,draw,2229,2220,C54,17,7970.161,45,45
20015,True,42,resign,black,2162,2214,E11,8,1347.822,7,30
20016,True,102,resign,black,2236,2148,B06,2,3439.334,7,30
20017,True,97,resign,white,2268,2254,C59,17,6704.261,60,30


In [ ]:
#Ora ci occupiamo di creare dummy per le colonne con valori categoriali: victory_status, winner e opening_eco
colonne_categoriali = ['winner', 'victory_status', 'opening_eco']

df = pd.get_dummies(df, columns=colonne_categoriali, dtype=int, drop_first=False)
df.tail()

,rated,turns,white_rating,black_rating,opening_ply,duration_seconds,base_time,increment_time,winner_black,winner_draw,...,opening_eco_E81,opening_eco_E87,opening_eco_E88,opening_eco_E90,opening_eco_E91,opening_eco_E92,opening_eco_E94,opening_eco_E95,opening_eco_E97,opening_eco_E98
20053,True,24,1691,1220,2,321.734,10,10,0,0,...,0,0,0,0,0,0,0,0,0,0
20054,True,82,1233,1196,2,744.219,10,0,1,0,...,0,0,0,0,0,0,0,0,0,0
20055,True,35,1219,1286,3,172.834,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20056,True,109,1360,1227,4,946.699,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20057,True,78,1235,1339,3,736.699,10,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.head()

,rated,turns,white_rating,black_rating,opening_ply,duration_seconds,base_time,increment_time,winner_black,winner_draw,...,opening_eco_E81,opening_eco_E87,opening_eco_E88,opening_eco_E90,opening_eco_E91,opening_eco_E92,opening_eco_E94,opening_eco_E95,opening_eco_E97,opening_eco_E98
0,False,13,1500,1191,5,0.0,15,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,True,16,1322,1261,4,0.0,5,10,1,0,...,0,0,0,0,0,0,0,0,0,0
2,True,61,1496,1500,3,0.0,5,10,0,0,...,0,0,0,0,0,0,0,0,0,0
3,True,61,1439,1454,3,0.0,20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,True,95,1523,1469,5,0.0,30,3,0,0,...,0,0,0,0,0,0,0,0,0,0
